In [0]:
# William Tandio
# Assignment 1

# Creating DBMS Folder
dbutils.fs.mkdirs("FileStore/tables/assignment1")
dbutils.fs.cp("FileStore/tables/geoPoints0.csv","FileStore/tables/assignment1")
dbutils.fs.cp("FileStore/tables/geoPoints1.csv","FileStore/tables/assignment1")

Out[2]: True

In [0]:
# Trying with Brute Force
sc = spark.sparkContext

data = sc.textFile("FileStore/tables/assignment1")

# extract the point, x and y values
data = data.map(lambda line: line.split(","))
data = data.map(lambda x: (x[0], float(x[1]), float(x[2])))

# euclidean distance formula
def calc_distance(pt1, pt2):
    return ((pt1[1]-pt2[1])**2+(pt1[2]-pt2[2])**2)**0.5

# find close pairs
def close_pairs(pointa, pointb, threshold):
    close_pairs = []
    for i in pointb:
        distance = calc_distance(pointa, i)
        if distance <= threshold: 
            close_pairs.append((pointa[0], i[0]))
    return close_pairs

# threshold value can be adjusted
threshold = 0.75

# get all close pairs
#get all possible combination
cartesian = data.cartesian(data)
#removing the duplicates
result = cartesian.filter(lambda x: x[0][0] < x[1][0])
#applying the function into the rdd
#print(result.collect())
finalresult = result.flatMap(lambda x: close_pairs(x[0], [x[1]], threshold))


# convert the close pairs to a list
close_pairs_list = finalresult.collect()
print('Brute Force with Cartesian')
print(len(close_pairs_list), close_pairs_list)

#Brute Force with Loop
sc = spark.sparkContext

data = sc.textFile("FileStore/tables/assignment1")

# extract the point, x and y values
data = data.map(lambda line: line.split(","))
data = data.map(lambda x: (x[0], float(x[1]), float(x[2])))
points_list = data.collect()

def euclidean_distance(pt1, pt2):
    return ((pt1[1]-pt2[1])**2+(pt1[2]-pt2[2])**2)**0.5

def calculate_distances(points_list, threshold):
    distances = []
    for i in range(len(points_list)):
        for j in range(i+1, len(points_list)): 
            distance = euclidean_distance(points_list[i], points_list[j])
            if distance < threshold:
                distances.append((points_list[i][0], points_list[j][0]))
    return distances

threshold = 0.75
result = calculate_distances(points_list, threshold)
print('Brute Force with For Loop')
print(len(result),result)

Brute Force with Cartesian
8 [('Pt05', 'Pt07'), ('Pt06', 'Pt09'), ('Pt01', 'Pt14'), ('Pt01', 'Pt15'), ('Pt03', 'Pt10'), ('Pt05', 'Pt15'), ('Pt07', 'Pt15'), ('Pt11', 'Pt17')]
Brute Force with For Loop
8 [('Pt01', 'Pt14'), ('Pt01', 'Pt15'), ('Pt03', 'Pt10'), ('Pt05', 'Pt07'), ('Pt05', 'Pt15'), ('Pt06', 'Pt09'), ('Pt07', 'Pt15'), ('Pt11', 'Pt17')]


In [0]:
# Forming Grid Method
sc = spark.sparkContext

data = sc.textFile("FileStore/tables/assignment1")

# extract the point, x and y values
data = data.map(lambda line: line.split(","))
data = data.map(lambda x: (x[0], float(x[1]), float(x[2])))

# threshold distance
threshold_distance = 0.75

# grid cell size
cell_size = 0.75


########################### FORMING GRID ##############################
# Assign each point to a grid cell
def grid_cell(point):
    x, y = point[1], point[2]
    i = int(x / cell_size)
    j = int(y / cell_size)
    return (i, j), (point[0], x, y)

#keys = gridcell.keys().collect()
def push_neighbor(cells):
    # Push is a list of where all my points are going to be pushed
    push = []
    #Storing all neighbors value
    neighbors = []
    cell, val = cells[0], cells[1]
    i, j = cell
    #Creating neighbor list
    neighbors.append((i-1,j-1))
    neighbors.append((i,j-1))
    neighbors.append((i+1,j-1))
    neighbors.append((i+1,j))
    #Creating the push list
    for i in neighbors:
        #Only push to a non-empty cell
        x = (i,val)
        push.append(x)
    #Handling duplicates
    # If the cell had more than 1 point and have a neighbor then push and leave the cell empty
    # since if its in the same cells, its going to be calculated again in the neighbor cell
    if len(val)>1 and push != []:
        return push
    else:
        #If there is no neighbor to push, then stay at its original cell and push
        push.append(cells)
        return push
    

#Create RDD with cell as keys and point information as values, group it so each key have points in 1 cell
#the mapValues is only to show the value for print
gridcell = data.map(grid_cell)
print(gridcell.collect())


pushcell = gridcell.flatMap(lambda x: (push_neighbor(x)))
#print(pushcell.collect())

# Groupbykey
groupedcell = pushcell.groupByKey().mapValues(list)
#print(groupedcell.collect())


#################### CALCULATION PART #######################

#Distance Formula
def euclidean_distance(pt1, pt2):
    return ((pt1[1]-pt2[1])**2+(pt1[2]-pt2[2])**2)**0.5

def calc_distances(points_list):
    distances = []
    for i in range(len(points_list)):
        for j in range(i+1, len(points_list)): 
            distance = euclidean_distance(points_list[i], points_list[j])
            if distance < threshold_distance:
                distances.append((points_list[i][0], points_list[j][0]))
    return distances


#result = groupedcell.map(lambda x:(calc_distances(x[1])))
result = groupedcell.flatMap(lambda x:(calc_distances(x[1])))
#print(result.count(),result.collect())



[((4, 0), ('Pt00', 3.49838, 0.662006)), ((4, 4), ('Pt01', 3.0474, 3.20142)), ((1, 6), ('Pt02', 1.49829, 5.05809)), ((4, 6), ('Pt03', 3.55013, 4.96649)), ((6, 5), ('Pt04', 4.84535, 4.05086)), ((5, 4), ('Pt05', 3.88558, 3.55233)), ((0, 2), ('Pt06', 0.0127773, 2.04053)), ((4, 5), ('Pt07', 3.53014, 3.80594)), ((5, 7), ('Pt08', 4.15061, 5.80679)), ((0, 2), ('Pt09', 0.00945363, 1.83433)), ((4, 6), ('Pt10', 3.18407, 4.52642)), ((0, 0), ('Pt11', 0.68422, 0.734435)), ((7, 3), ('Pt12', 5.31465, 2.48444)), ((4, 3), ('Pt13', 3.37496, 2.49623)), ((3, 4), ('Pt14', 2.75711, 3.04256)), ((4, 4), ('Pt15', 3.7311, 3.22135)), ((5, 2), ('Pt16', 3.93387, 1.98305)), ((1, 0), ('Pt17', 1.05066, 0.355635)), ((6, 2), ('Pt18', 4.99483, 1.53335)), ((3, 1), ('Pt19', 2.33941, 0.867107))]


In [0]:
def get_neighbors(cell):
    neighbors = []
    i, j = cell
    for x in range(i-1, i+2):
        for y in range(j-1, j+2):
            if (x, y) != (i, j): #Ignore same cell
                if x >= 0 and y >= 0: # ignore negative values
                    neighbors.append((x, y))
    return cell,neighbors

test = gridcell.map(lambda x: (x[0],x[1],get_neighbors(x[0])))
print(test.collect())

[((4, 0), ('Pt00', 3.49838, 0.662006), ((4, 0), [(3, 0), (3, 1), (4, 1), (5, 0), (5, 1)])), ((4, 4), ('Pt01', 3.0474, 3.20142), ((4, 4), [(3, 3), (3, 4), (3, 5), (4, 3), (4, 5), (5, 3), (5, 4), (5, 5)])), ((1, 6), ('Pt02', 1.49829, 5.05809), ((1, 6), [(0, 5), (0, 6), (0, 7), (1, 5), (1, 7), (2, 5), (2, 6), (2, 7)])), ((4, 6), ('Pt03', 3.55013, 4.96649), ((4, 6), [(3, 5), (3, 6), (3, 7), (4, 5), (4, 7), (5, 5), (5, 6), (5, 7)])), ((6, 5), ('Pt04', 4.84535, 4.05086), ((6, 5), [(5, 4), (5, 5), (5, 6), (6, 4), (6, 6), (7, 4), (7, 5), (7, 6)])), ((5, 4), ('Pt05', 3.88558, 3.55233), ((5, 4), [(4, 3), (4, 4), (4, 5), (5, 3), (5, 5), (6, 3), (6, 4), (6, 5)])), ((0, 2), ('Pt06', 0.0127773, 2.04053), ((0, 2), [(0, 1), (0, 3), (1, 1), (1, 2), (1, 3)])), ((4, 5), ('Pt07', 3.53014, 3.80594), ((4, 5), [(3, 4), (3, 5), (3, 6), (4, 4), (4, 6), (5, 4), (5, 5), (5, 6)])), ((5, 7), ('Pt08', 4.15061, 5.80679), ((5, 7), [(4, 6), (4, 7), (4, 8), (5, 6), (5, 8), (6, 6), (6, 7), (6, 8)])), ((0, 2), ('Pt09', 0

In [0]:
cells = (1, 6), ('Pt02', 1.49829, 5.05809)
def get_neighbor(cells):
    keys = [(4, 0), (4, 4), (1, 6), (4, 6), (6, 5), (5, 4), (0, 2), (4, 5), (5, 7), (0, 2), (4, 6), (0, 0), (7, 3), (4, 3), (3, 4), (4, 4), (5, 2), (1, 0), (6, 2), (3, 1)]
    push = []
    neighbors = []
    cell, val = cells[0], cells[1]
    i, j = cell
    for x in range(i-1, i+2):
        for y in range(j-1, j+2):
            if (x, y) != (i, j): #Ignore same cell
                if x >= 0 and y >= 0: # ignore negative values
                    neighbors.append((x, y))
    print(neighbors)
    for i in neighbors:
        if i in keys:
            x = (i,val)
            push.append(x)
    if push == []:
        return cells
    else:
        return push
            
    
get_neighbor(cells)

[(0, 5), (0, 6), (0, 7), (1, 5), (1, 7), (2, 5), (2, 6), (2, 7)]
Out[1]: ((1, 6), ('Pt02', 1.49829, 5.05809))

In [0]:
cell=(2,5)
i, j = cell
neighbors = []
neighbors.append((i-1,j-1))
neighbors.append((i,j-1))
neighbors.append((i+1,j-1))
neighbors.append((i+1,j))
                    
print(neighbors)


def push_neighbor(cells):
    #Keys are location of all other points
    keys = [(4, 0), (4, 4), (1, 6), (4, 6), (6, 5), (5, 4), (0, 2), (4, 5), (5, 7), (0, 2), (4, 6), (0, 0), (7, 3), (4, 3), (3, 4), (4, 4), (5, 2), (1, 0), (6, 2), (3, 1)]
    # Push is a list of where all my points are going to be pushed, initial value cells is itself
    push = [cells]
    #Storing all neighbors value
    neighbors = []
    cell, val = cells[0], cells[1]
    i, j = cell
    #Creating neighbor list
    neighbors.append((i-1,j-1))
    neighbors.append((i,j-1))
    neighbors.append((i+1,j-1))
    neighbors.append((i+1,j))
    #Creating the push list
    for i in neighbors:
        if i in keys:
            x = (i,val)
            push.append(x)
    return push

[(1, 4), (2, 4), (3, 4), (3, 5)]


In [0]:
# Forming Grid Method
sc = spark.sparkContext

data = sc.textFile("FileStore/tables/assignment1")

# extract the point, x and y values
data = data.map(lambda line: line.split(","))
data = data.map(lambda x: (x[0], float(x[1]), float(x[2])))

# threshold distance
threshold_distance = 0.75

# grid cell size
cell_size = 0.75


########################### FORMING GRID ##############################
# Assign each point to a grid cell
def grid_cell(point):
    x, y = point[1], point[2]
    i = int(x / cell_size)
    j = int(y / cell_size)
    return (i, j), (point[0], x, y)


#Neighbor Function, grabbing all cells in the direction of "down right", including its cell
def neighbor(cell):
    neighbors = [cell]
    i, j = cell
    #Creating neighbor list
    neighbors.append((i-1,j-1))
    neighbors.append((i,j-1))
    neighbors.append((i+1,j-1))
    neighbors.append((i+1,j))
    return neighbors
    
#Create RDD with cell as keys and point information as values, group it so each key have points in 1 cell
#the mapValues is only to show the value for print
gridcell = data.map(grid_cell)
#print(gridcell.collect())

#The format is (All of the neighbors list),((Original Node),(Point1,X,Y))
neighborcell = gridcell.map(lambda x: (neighbor(x[0]),x))
#print(neighborcell.collect())

#Push the neighbor looping the neighbors list and pair it with the cell, where each cell also consist of its home node
flatneighbor = neighborcell.flatMap(lambda x: ((i,x[1]) for i in x[0]))
#print(flatneighbor.collect())

# Groupbykey
groupedcell = flatneighbor.groupByKey().mapValues(list)
#print(groupedcell.collect())


#################### CALCULATION PART #######################

#Distance Formula
def euclidean_distance(cell1, cell2):
    pt1 = cell1[1]
    pt2 = cell2[1]
    print(pt1,pt2)
    dist = ((pt1[1]-pt2[1])**2+(pt1[2]-pt2[2])**2)**0.5
    return dist

#Cells in this function is the whole information of ((Current Node being calculated),[List of all cell that are being pushed into this node with their original home node])
#Example "Cells": ((4, 0), [((4, 0), ('Pt00', 3.49838, 0.662006)), ((3, 1), ('Pt19', 2.33941, 0.867107))])
def calc_distances(cells):
    current_node = cells[0]
    points_list = cells[1]
    distances = []
    for i in range(len(points_list)):
        for j in range(i+1, len(points_list)): 
            home_node1 = points_list[i][0]
            home_node2 = points_list[j][0]
            #Calculate only if neither of the node if from home node.
            #Avoid duplicates that calculate points that are not from this node
            if home_node1 == current_node or home_node2 == current_node:
                distance = euclidean_distance(points_list[i], points_list[j])
                if distance < threshold_distance:
                    distances.append((points_list[i][1][0], points_list[j][1][0]))
                    print('append!')          
    return distances


#result = groupedcell.map(lambda x:(calc_distances(x[1])))
result = groupedcell.flatMap(lambda x:(calc_distances(x)))
print(result.count(),result.collect())

8 [('Pt01', 'Pt14'), ('Pt01', 'Pt15'), ('Pt07', 'Pt15'), ('Pt03', 'Pt10'), ('Pt06', 'Pt09'), ('Pt05', 'Pt07'), ('Pt05', 'Pt15'), ('Pt11', 'Pt17')]


In [0]:
def euclidean_distance(cell1, cell2):
    pt1 = cell1[1]
    pt2 = cell2[1]
    print(pt1,pt2)
    dist = ((pt1[1]-pt2[1])**2+(pt1[2]-pt2[2])**2)**0.5
    return dist

def calc_distances(cells):
    current_node = cells[0]
    points_list = cells[1]
    distances = []
    for i in range(len(points_list)):
        for j in range(i+1, len(points_list)): 
            home_node1 = points_list[i][0]
            home_node2 = points_list[j][0]
            print('c=',current_node,'h1=', home_node1,'h2=', home_node2)
            if home_node1 == current_node or home_node2 == current_node:
                distance = euclidean_distance(points_list[i], points_list[j])
                if distance < threshold_distance:
                    distances.append((points_list[i][1][0], points_list[j][1][0]))
                    print('append!')     
    return distances

x = ((4, 0), [((4, 0), ('Pt00', 3.49838, 0.662006)), ((3, 1), ('Pt19', 2.33941, 0.867107))]), ((4, -1), [((4, 0), ('Pt00', 3.49838, 0.662006))])
calc_distances(x)

c= ((4, 0), [((4, 0), ('Pt00', 3.49838, 0.662006)), ((3, 1), ('Pt19', 2.33941, 0.867107))]) h1= 4 h2= ((4, 0), ('Pt00', 3.49838, 0.662006))
Out[157]: []